<a href="https://colab.research.google.com/github/MinsooKwak/RAG/blob/main/test/deploy/RAG_paper_assistant_chatbot_for_deploy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install cohere
#!pip install langchain
#!pip install openai
#!pip install -U langchain-community
#!pip install pypdf
#!pip install tiktoken
#!pip install faiss-cpu
#!pip install -U langchain-openai
#!pip install streamlit

In [ ]:
from langchain.document_loaders import PyPDFLoader

# PDF 가져오기

loader = PyPDFLoader("2306.05685v4.pdf")

"""
loaders = [
    PyPDFLoader("2306.05685v4.pdf")
]

docs = []
for loader in loaders:
  docs.extend(loader.load())
"""

data = loader.load()

In [ ]:
import os
from config import OPEN_AI_API_KEY, COHERE_API_KEY, NGROK_TOKEN_KEY
os.environ["COHERE_API_KEY"] = COHERE_API_KEY
os.environ["OPENAI_API_KEY"] = OPEN_AI_API_KEY
os.environ["NGROK_TOKEN"] = NGROK_TOKEN_KEY

Chunking

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 500,
    chunk_overlap  = 50,
)

data = text_splitter.split_documents(data)

Embedding

In [ ]:
#from langchain.embeddings import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", api_key=OPEN_AI_API_KEY)

FAISS database

In [ ]:
from langchain.vectorstores import FAISS
vectorstore = FAISS.from_documents(data, embeddings)

LLM

- memory 부분에 대화 내용도 함께 넘어가므로 token size가 중요하다

In [ ]:
#from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo", api_key=OPEN_AI_API_KEY)

In [ ]:
# 대화 내용 기억하도록
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

In [ ]:
from langchain.chains import ConversationalRetrievalChain
conversational_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),     # 검색 결과 넘겨줌
    memory=memory                             # 대화 내용도 같이 전달 (token따라 구현x)
)

In [ ]:
query = "기존에 활용된 평가 방식과는 어떤 차이점이 있어?"
result = conversational_chain({"question": query})  # 질문 넘겨줌
answer = result["answer"]
answer

<ipython-input-10-579eb2af6986>:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  result = conversational_chain({"question": query})  # 질문 넘겨줌


'이 논문에서는 기존 평가 방식과 비교하여 LLM-as-a-judge 방식의 잠재적 한계를 연구하고 있습니다. 이에 대한 몇 가지 예상한 한계는 위치 편향, 장황 편향, 자아 강화 편향 및 제한된 추론 능력입니다. 이러한 한계들은 인간 평가의 골드 표준과 비교하여 분석되고 있습니다.'

In [ ]:
query = "LLM-as-a-judge 방법은 어떻게 작동이 돼?"
result = conversational_chain({"question": query})  # 질문 넘겨줌
answer = result["answer"]
answer

'LLM-as-a-judge 방법은 Large Language Models (LLMs)를 판단 기준으로 사용하여 텍스트나 대화 모델의 성능을 평가하는 방법입니다. 이 방법은 LLM이 사람과의 대조를 통해 응답을 판단하고 사람의 선호도와 일치하는지 확인하는 것을 목표로 합니다. LLM-as-a-judge의 세 가지 변형이 제안되었으며, 이는 독립적으로 또는 결합하여 구현될 수 있습니다. 이러한 방법은 여러 가지 편향과 한계를 가지고 있지만, 해결책을 제안하고 이러한 한계를 극복하는 방법을 연구하고 있습니다.'

In [ ]:
query = "변형에 대해 자세히 알려줘."
result = conversational_chain({"question": query})  # 질문 넘겨줌
answer = result["answer"]
answer

'LLM-as-a-judge 방법의 세 가지 변형은 다음과 같습니다:\n\n1. Position bias: LLM이 특정 위치를 다른 위치보다 선호하는 경향을 나타내는 바이어스입니다. 이러한 편향은 인간의 의사결정에서도 볼 수 있으며, 다른 기계 학습 분야에서도 관측되었습니다.\n\n2. Verbosity bias: LLM이 말이 많은 응답을 선호하는 경향을 나타내는 바이어스입니다.\n\n3. Self-enhancement bias: LLM이 자아 강화를 위한 성향을 나타내는 바이어스입니다.\n\n이러한 세 가지 변형에 대한 제한 사항과 해결책에 대해 논의되고 있습니다.'

In [ ]:
query = "제시한 방법이 어떤 장점이 있고 어떤 한계가 있는지 알려줘."
result = conversational_chain({"question": query})  # 질문 넘겨줌
answer = result["answer"]
answer

'LLM-as-a-judge의 장점은 확장성과 설명가능성입니다. 이는 인간의 개입을 줄여 확장 가능한 벤치마크와 빠른 반복을 가능케 합니다. 게다가 LLM 판정자는 점수 뿐만 아니라 설명도 제공하여 그 결과를 해석하기 쉽게 만들어줍니다. \n\n한편 LLM-as-a-judge의 한계는 위치 편향(position bias)과 같은 편향이 있습니다. 이러한 편향은 우리의 맥락에서만 발생하는 것이 아니며, 인간의 의사결정에서도 보여지고 다른 기계학습 분야에서도 관찰되었습니다. 이러한 한계들에도 불구하고, 나중에 해결책을 제시하고 LLM 판정자와 인간 사이의 일치가 높음을 보여줄 것입니다.'

In [4]:
!pip install pyngrok
from pyngrok import ngrok

- ngrok 사이트 : https://dashboard.ngrok.com/signup

In [ ]:
!rm -rf ~/.streamlit/cache

In [ ]:
# 캐시 삭제
import streamlit as st
st.cache_data.clear()

2024-09-07 14:51:39.881 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


- rag 미활용 app

In [21]:
%%writefile app.py
import os
import streamlit as st
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferWindowMemory
import tempfile  # 임시 파일 생성에 사용

# 캐시 삭제
st.cache_data.clear()

# LLM Prompt 정의
prompt = PromptTemplate(
    input_variables=["combined_input"],
    template="""
    You are an AI assistant based on research papers.
    Answer the following question based on the content of the provided papers.

    {combined_input}

    AI:"""
)

# OpenAI 모델과 메모리 설정
llm = ChatOpenAI(model_name='gpt-4')  # 또는 'gpt-3.5-turbo' 사용 가능
memory = ConversationBufferWindowMemory(memory_key='chat_history', k=4)  # 최근 4개의 대화만 기억

llm_chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt
)

# Create embeddings
embeddings = OpenAIEmbeddings()

st.title("ChatGPT AI Assistant")
st.write("논문 파일들을 입력해주세요.")

# 여러 PDF 파일 업로드 기능 추가
uploaded_files = st.file_uploader("논문 PDF 파일들을 업로드하세요", type=["pdf"], accept_multiple_files=True)

if uploaded_files:
    all_text = ""

    for uploaded_file in uploaded_files:
        with tempfile.NamedTemporaryFile(delete=False) as temp_file:
            temp_file.write(uploaded_file.read())
            temp_file_path = temp_file.name

        # PyPDFLoader로 임시 파일을 로드
        loader = PyPDFLoader(temp_file_path)
        documents = loader.load()
        pdf_text = "\n".join([doc.page_content for doc in documents])
        all_text += pdf_text + "\n"

    st.write("### 업로드한 논문들 내용 미리보기:")
    st.write(all_text[:1000])

    if "messages" not in st.session_state.keys():
        st.session_state.messages = [
            {"role": "system", "content": "안녕하세요, 저는 AI Assistant입니다. 업로드한 논문들을 바탕으로 질문에 답변해 드립니다."}
        ]

    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.write(message['content'])

    user_prompt = st.chat_input("논문에 대한 질문을 입력하세요...")

    if user_prompt and user_prompt.strip():
        st.session_state.messages.append({"role": "user", "content": user_prompt})
        with st.chat_message("user"):
            st.write(user_prompt)

        if st.session_state.messages[-1]["role"] != "assistant":
            with st.chat_message("assistant"):
                with st.spinner("Thinking..."):
                    try:
                        if all_text.strip():  # 논문 내용이 비어 있지 않도록 확인
                            # 질문과 논문 내용을 하나의 텍스트로 결합
                            combined_input = f"Question: {user_prompt}\n\nPapers content:\n{all_text}"

                            # llm_chain.predict()로 단일 입력값 전달
                            ai_response = llm_chain.predict(combined_input=combined_input)

                            if ai_response and ai_response.strip():
                                st.write(ai_response)
                                new_ai_message = {"role": "assistant", "content": ai_response}
                                st.session_state.messages.append(new_ai_message)
                            else:
                                st.write("AI가 응답하지 못했습니다. 다시 시도해주세요.")
                        else:
                            st.write("업로드된 논문에서 내용을 찾을 수 없습니다.")
                    except Exception as e:
                        st.error(f"오류 발생: {str(e)}")
    else:
        st.write("질문을 입력해주세요.")

Writing app.py


- rag 활용 app for deploy

In [25]:
%%writefile rag_paper_chat_assistant_app.py
import os
import streamlit as st
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferWindowMemory
import tempfile  # 임시 파일 생성에 사용

# 캐시 삭제
st.cache_data.clear()

# LLM Prompt 정의
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
    You are an AI assistant. Based on the research papers provided, answer the following question.

    Context: {context}
    Question: {question}

    AI:"""
)

# 사용자로부터 API키 입력 받음
api_key = st.text_input("OpenAI API 키를 입력하세요:", type="password")

if api_key:
    os.environ["OPENAI_API_KEY"] = api_key

    # openai 모델 초기화
    llm = ChatOpenAI(model_name="gpt-4", api_key=api_key)
    memory = ConversationBufferWindowMemory(memory_key='chat_history', k=4)

    # Create embeddings
    embeddings = OpenAIEmbeddings()

  st.title("ChatGPT AI Assistant with RAG")

  #### 여러 PDF 파일 업로드 기능 추가
  uploaded_files = st.file_uploader("논문 PDF 파일들을 업로드하세요", type=["pdf"], accept_multiple_files=True)


  if uploaded_files:
      all_text = ""
      documents = []

      for uploaded_file in uploaded_files:
          with tempfile.NamedTemporaryFile(delete=False) as temp_file:
              temp_file.write(uploaded_file.read())
              temp_file_path = temp_file.name

          # PyPDFLoader로 임시 파일을 로드
          loader = PyPDFLoader(temp_file_path)
          docs = loader.load()
          documents.extend(docs)
          # 업로드 문서 보기 위한 작업
          pdf_text = "\n".join([doc.page_content for doc in documents])
          all_text += pdf_text + "\n"

      st.write("### 업로드한 논문들 내용 미리보기:")
      st.write(all_text[:1000])

  #### Chunking
      text_splitter = RecursiveCharacterTextSplitter(
          chunk_size=750,
          chunk_overlap=50,
          separators=["\n\n"]
      )

      chunks = text_splitter.split_documents(documents)

      # FAISS vectorDB 생성
      vectorstore = FAISS.from_documents(chunks, embeddings)

      #### Chains
      # RetrievalQA chain을 사용할 때
      qa_chain = RetrievalQA.from_chain_type(
          llm=llm,
          chain_type="stuff",  # "stuff" simply concatenates the retrieved chunks
          retriever=vectorstore.as_retriever(),
          return_source_documents=True  # This returns the documents that were retrieved as well
      )

      # ConversationalRetrievalChain을 사용할 때
      conversational_chain = ConversationalRetrievalChain.from_llm(
          llm=llm,
          chain_type="stuff",
          retriever=vectorstore.as_retriever(),     # 검색 결과 넘겨줌
          #memory=memory                            # 대화 내용도 같이 전달 (token 소비에 따라 구현x)
      )

      if "messages" not in st.session_state.keys():
          st.session_state.messages = [
              {"role": "system", "content": "안녕하세요, 저는 AI Assistant입니다. 업로드한 논문들을 바탕으로 질문에 답변해 드립니다."}
          ]

      for message in st.session_state.messages:
          with st.chat_message(message["role"]):
              st.write(message['content'])

      user_prompt = st.chat_input("논문에 대한 질문을 입력하세요...")

      if user_prompt and user_prompt.strip():
          st.session_state.messages.append({"role": "user", "content": user_prompt})
          with st.chat_message("user"):
              st.write(user_prompt)

          if st.session_state.messages[-1]["role"] != "assistant":
              with st.chat_message("assistant"):
                  with st.spinner("Retrieving relevant sections..."):
                      try:
                          if chunks:
                              #response = qa_chain({"query": user_prompt})                  # qa_chain 사용할 때
                              response = conversational_chain({"question": user_prompt})    # conversational_chain 사용할 때

                              ai_response = response["result"]
                              source_docs = response["source_documents"]

                              if ai_response and ai_response.strip():
                                  st.write(ai_response)
                                  st.write("\n참조된 문서들:")
                                  for doc in source_docs:
                                      st.write(doc.page_content[:500])                      # 참조된 doc의 첫 500자까지 보여줌

                                  new_ai_message = {"role": "assistant", "content": ai_response}
                                  st.session_state.messages.append(new_ai_message)
                              else:
                                  st.write("AI가 응답하지 못했습니다. 다시 시도해주세요.")
                          else:
                              st.write("업로드된 논문에서 내용을 찾을 수 없습니다.")
                      except Exception as e:
                          st.error(f"오류 발생: {str(e)}")
      else:
          st.write("질문을 입력해주세요.")
else:
    st.warning("API 키를 입력해야 논문 assistant 사용이 가능합니다.")

Overwriting rag_paper_chat_assistant_app.py


In [26]:
# 현재 실행 중인 모든 터널 종료
ngrok.kill()

- rag 활용 앱 실행

In [27]:
import os
from config import OPEN_AI_API_KEY, COHERE_API_KEY, NGROK_TOKEN_KEY
os.environ["NGROK_TOKEN"] = NGROK_TOKEN_KEY

In [28]:
ngrok.set_auth_token(os.environ["NGROK_TOKEN"])

!streamlit run rag_paper_chat_assistant_app.py &>/dev/null&
public_url = ngrok.connect(8501, "http")
public_url

<NgrokTunnel: "https://f66d-34-172-211-3.ngrok-free.app" -> "http://localhost:8501">